In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# default_exp optuna_objective

# Optuna Objective

> An API to create Optuna Ojbective

In [3]:
#hide
from nbdev.showdoc import *
from nbdev import *

In [4]:
# export
from tabular_ml_toolkit.dataframeloader import *
from tabular_ml_toolkit.preprocessor import *
from tabular_ml_toolkit.logger import *

In [5]:
# export
# hide
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, roc_auc_score,accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
# for Optuna
import optuna
#for XGB
from xgboost import XGBRegressor, XGBClassifier


# for displaying diagram of pipelines 
from sklearn import set_config
set_config(display="diagram")

# for finding n_jobs in all sklearn estimators
from sklearn.utils import all_estimators
import inspect

# Just to compare fit times
import time

# for os specific settings
import os

In [6]:
# export

class Optuna_Objective:
    """
    Represent Optuna Objective class
    
    Attributes:\n
    pipeline: An MLPipeline instance \n
    dfl: A DataFrameLoader instance \n
    pp: A PreProcessor Instance \n
    model: The given Model
    """

    def __init__(self, dfl, tmlt, task, xgb_eval_metric, kfold_splits, kfold_metrics, use_gpu):
        self.X = dfl.X
        self.y = dfl.y
        self.task = task
        self.use_gpu = use_gpu
        self.tmlt = tmlt
        self.xgb_model = None
        self.trial = None
        self.xgb_eval_metric = xgb_eval_metric
        self.kfold_splits = kfold_splits
        self.kfold_metrics = kfold_metrics

    def __call__(self, trial):
        x, y = self.X, self.y
        self.trial = trial
        
        #get_params here
        self.xgb_params = self.get_xgb_params(self.trial, use_gpu=self.use_gpu)
        early_stopping_rounds = self.xgb_params["early_stopping_rounds"]
        del self.xgb_params["early_stopping_rounds"]

        # get xgb model based on task type
        if self.task == "regression":
            self.xgb_model = XGBRegressor(eval_metric=self.xgb_eval_metric,random_state=42,
                                          use_label_encoder=False,**self.xgb_params)
        if self.task == "classification":
            self.xgb_model = XGBClassifier(eval_metric=self.xgb_eval_metric,random_state=42,
                                           use_label_encoder=False,**self.xgb_params)

        # update the model here on tmlt pipeline
        self.tmlt.update_model(self.xgb_model)

        #rest remains same
        score, _ = self.tmlt.do_k_fold_training(n_splits=self.kfold_splits, metrics=self.kfold_metrics)
        metrics_mean_score = np.mean(score)
        return metrics_mean_score
                
        
    
    def __str__(self):
        """Returns human readable string reprsentation"""
        attr_str = ("X, y, task, use_gpu, tmlt, xgb_model")
        return ("Training Pipeline object with attributes:"+attr_str)
    
    def __repr__(self):
        return self.__str__()

    #helper methods
    #Method to get xgb_params
    def get_xgb_params(self, trial, use_gpu):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 100.0, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 100.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.1, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
            "max_depth": trial.suggest_int("max_depth", 1, 9),
            "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 100, 500),
            "n_estimators": trial.suggest_categorical("n_estimators", [7000, 15000, 20000]),
        }
        if use_gpu:
            params["tree_method"] = "gpu_hist"
            params["gpu_id"] = 0
            params["predictor"] = "gpu_predictor"
        else:
            params["tree_method"] = trial.suggest_categorical("tree_method", ["exact", "approx", "hist"])
            params["booster"] = trial.suggest_categorical("booster", ["gbtree", "gblinear"])
            if params["booster"] == "gbtree":
                params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
                params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

        return params
                
    # core methods

In [7]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()

Converted 00_dataframeloader.ipynb.
Converted 01_preprocessor.ipynb.
Converted 02_mlpipeline.ipynb.
Converted 03_tutorial.ipynb.
Converted 04_optuna_objective.ipynb.
Converted 07_kaggle_tps_tutorial.ipynb.
Converted 08_optuna_tutorial.ipynb.
Converted do_optuna_opt_tutorial.ipynb.
Converted index.ipynb.
Converted logger.ipynb.
